In [1]:
from intransparent import (
    YEAR_LABELS,
    ingest_reports_per_country,
    without_populations,
    reports_per_capita_country_year,

    REPORTS_PER_PLATFORM,
    ingest_reports_per_platform,
    compare_all_platform_reports,

    show_html,
    show_info,
    show_table,
)

import intransparent.meta as meta

show_html(h1='CSAM Reports per Country', h2='Prelude')

country_data = ingest_reports_per_country('./data', logger=show_info)

countries_without, reports_without = (
    without_populations(country_data.reports, country_data.populations))
show_html(
    f'{countries_without.shape[0]} countries with reports have no population '
    'statistics. They also account for very few reports.')
show_table(reports_without, title='Reports of countries without population')


,reports,reports_pct
year,,
2019,28,0.000165
2020,97,0.000446
2021,243,0.000827
2022,117,0.000365


In [2]:
show_html(
    h2='Regions with Most CSAM Reports', 
    p='Selection does <em>not</em> take population counts into consideration.')

most_reports = (
    country_data.reports_per_capita
    .groupby(['year', 'region'])
    .sum(numeric_only=True)
    .sort_values(by='reports', ascending=False)
)

for year in YEAR_LABELS:
    show_table(
        most_reports.query(f'year == "{year}"').head(20),
        title=f'Regions with most CSAM Reports {year}',
        highlights=['reports', 'reports_pct'],
    )


In [3]:
show_html(h2='Countries with Most CSAM Reports Per Capita')

for year, group in reports_per_capita_country_year(country_data):
    show_table(
        group.head(20),
        title=f'reports per capita and country {year}',
        highlights='reports_per_capita',
    )


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country
rank,,,,,,,,
1,ARE,2019,"330,268",1.944198,"9,211,657",0.118631,0.035853,United Arab Emirates
2,IRQ,2019,"1,026,809",6.044547,"41,563,521",0.535271,0.024705,Iraq
3,LBY,2019,"150,848",0.888001,"6,569,088",0.084599,0.022963,Libya
4,QAT,2019,"53,467",0.314746,"2,807,235",0.036153,0.019046,Qatar
5,BHR,2019,"26,093",0.153602,"1,494,189",0.019243,0.017463,Bahrain
6,DZA,2019,"700,535",4.123860,"42,705,369",0.549976,0.016404,Algeria
7,SYC,2019,"1,501",0.008836,"104,373",0.001344,0.014381,Seychelles
8,SAU,2019,"514,832",3.030677,"35,827,362",0.461398,0.014370,Saudi Arabia
9,KWT,2019,"63,492",0.373760,"4,441,100",0.057194,0.014296,Kuwait


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country
rank,,,,,,,,
1,LBY,2020,"189,955",0.873313,"6,653,942",0.084861,0.028548,Libya
2,DZA,2020,"1,102,939",5.070731,"43,451,666",0.554163,0.025383,Algeria
3,MCO,2020,886,0.004073,"36,922",0.000471,0.023997,Monaco
4,ARE,2020,"216,874",0.997072,"9,287,289",0.118446,0.023352,United Arab Emirates
5,IRQ,2020,"919,966",4.229518,"42,556,984",0.542753,0.021617,Iraq
6,QAT,2020,"56,683",0.260598,"2,760,385",0.035205,0.020534,Qatar
7,BTN,2020,"15,379",0.070705,"772,506",0.009852,0.019908,Bhutan
8,MAR,2020,"705,863",3.245185,"36,688,772",0.467912,0.019239,Morocco
9,KWT,2020,"68,582",0.315304,"4,360,445",0.055611,0.015728,Kuwait


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country
rank,,,,,,,,
1,LBY,2021,"270,811",0.921199,"6,735,277",0.085156,0.040208,Libya
2,ARE,2021,"327,820",1.115122,"9,365,145",0.118407,0.035004,United Arab Emirates
3,IRQ,2021,"1,220,470",4.151586,"43,533,593",0.550411,0.028035,Iraq
4,PHL,2021,"3,188,793",10.847090,"113,880,328",1.439829,0.028001,Philippines
5,QAT,2021,"73,536",0.250142,"2,688,235",0.033988,0.027355,Qatar
6,DZA,2021,"1,171,653",3.985529,"44,177,969",0.558558,0.026521,Algeria
7,KHM,2021,"395,793",1.346341,"16,589,024",0.209741,0.023859,Cambodia
8,BHR,2021,"32,346",0.110029,"1,463,266",0.018501,0.022105,Bahrain
9,BLZ,2021,"8,297",0.028223,"400,031",0.005058,0.020741,Belize


,iso3,year,reports,reports_pct,population,population_pct,reports_per_capita,country
rank,,,,,,,,
1,ARE,2022,"265,307",0.827558,"9,441,129",0.118383,0.028101,United Arab Emirates
2,LBY,2022,"163,336",0.509485,"6,812,341",0.085420,0.023976,Libya
3,QAT,2022,"61,167",0.190795,"2,695,122",0.033794,0.022695,Qatar
4,PHL,2022,"2,576,182",8.035746,"115,559,009",1.448997,0.022293,Philippines
5,IRQ,2022,"905,883",2.825672,"44,496,122",0.557938,0.020359,Iraq
6,BHR,2022,"25,896",0.080776,"1,472,233",0.018460,0.017590,Bahrain
7,SAU,2022,"602,745",1.880110,"36,408,820",0.456531,0.016555,Saudi Arabia
8,DZA,2022,"731,167",2.280690,"44,903,225",0.563042,0.016283,Algeria
9,BLZ,2022,"6,103",0.019037,"405,273",0.005082,0.015059,Belize


In [4]:
import plotly.express as px

map_data = country_data.reports_per_capita.copy()
map_data = map_data.reset_index()

# The text for hover labels (without clunky hover data)
map_data['labels'] = (
    map_data['country'].astype(str)
    + ':<br>'
    + map_data['reports_per_capita'].apply(lambda v: f'{v:.5f}')
    + ' ('
    + map_data['year'].astype(str)
    + ')'
)

fig = px.choropleth(
    map_data,
    locations='iso3',
    color='reports_per_capita',
    color_continuous_scale=px.colors.sequential.Plasma_r,
    range_color=(0, 0.040),
    hover_name='labels',
    hover_data={'iso3': False, 'reports_per_capita': False, 'year': False},
    labels={'reports_per_capita': 'RPC'},
    animation_frame='year',
)
fig.update_geos(
    resolution=110,
    showframe=False,
    showocean=True,
    oceancolor='#eaeaec',
    showlakes=False,
    lataxis_range=[-58, 90], # Crop Antarctica out of map.
)
fig.update_layout(
    margin={'t': 0, 'r': 0, 'b': 0, 'l': 0},
    coloraxis=dict(
        #colorbar_len=0.76, # Only needed without animation, adjusts for crop
        colorbar_tickformat='.3f',
    ),
    # paper_bgcolor='#000', # Can be helpful when debugging size issues
)

show_html(
    h2='CSAM Reports Per Capita Per Country Per Year',
    p='With RPC standing for Reports Per Capita'
)
fig.show()

In [5]:
show_html(h2='CSAM Reports per Social Media Platform')

disclosures = ingest_reports_per_platform(
    REPORTS_PER_PLATFORM, include_redundant=True)
comparisons = compare_all_platform_reports(disclosures)
for platform, data in comparisons.items():
    show_table(data, title=platform)

,pieces,reports,Δ%,NCMEC
year,,,,
2020,,"3,432",0.000000,"3,432"
2021,"1,608","2,684",-14.940387,"2,283"
2022,"5,530","4,969",590.480982,"34,310"


,pieces,reports,Δ%,NCMEC
year,,,,
2019,,724,0.000000,724
2020,,"2,233",0.000000,"2,233"
2021,"9,258","10,059",0.000000,"10,059"
2022,"80,888","52,592",0.000000,"52,592"


,pieces,reports,Δ%,NCMEC
year,,,,
2019,,,,"82,030"
2020,,,,"144,095"
2021,,,,"512,522"
2022,"746,051","285,470",93.045154,"551,086"


,pieces,reports,Δ%,NCMEC
year,,,,
2020,"4,437,853","547,875",-0.213735,"546,704"
2021,"6,696,497","870,319",0.627816,"875,783"
2022,"13,402,885","2,174,319",0.010532,"2,174,548"


In [6]:
show_html(h2="Meta's Ever-Changing Transparency Disclosures")

meta_disclosures = meta.read_all('data', '2022q2', '2022q4')
meta_differences = meta.diff_all(meta_disclosures)

for p1, delta in meta_differences.items():
    divergent = (
        delta
        .groupby('period')
        .size()
        .to_frame()
        .rename(columns={0: 'divergent'})
    )

    p2 = p1 + 1
    title = f'Δ(Q{p1.quarter}-{p1.year} / Q{p2.quarter}-{p2.year})'
    show_table(divergent, title=title)
    show_html(meta.hype_divergent_descriptors(delta))

,divergent
period,
2020 Q4,77
2021 Q1,3
2021 Q2,4
2022 Q2,29


,divergent
period,
2020 Q2,67
2020 Q3,60
2020 Q4,58
2022 Q3,18
